<a href="https://colab.research.google.com/github/nahubn1/Hybrid-Robot-Navigation-System/blob/main/notebooks/results_analysis/Comparative_Inference_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Environment Setup
The following cells mount Google Drive, clone the project repository and install requirements.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
COLAB_PROJECT_ROOT = '/content/Thesis_Project'
if not os.path.exists(COLAB_PROJECT_ROOT):
  !git clone https://github.com/nahubn1/Hybrid-Robot-Navigation-System {COLAB_PROJECT_ROOT}
os.chdir(COLAB_PROJECT_ROOT) # Change directory into the project
!git pull # Ensure it's the latest version

Cloning into '/content/Thesis_Project'...
remote: Enumerating objects: 1201, done.
remote: Counting objects: 100% (256/256), done.
remote: Compressing objects: 100% (235/235), done.
remote: Total 1201 (delta 135), reused 21 (delta 21), pack-reused 945 (from 1)
Receiving objects: 100% (1201/1201), 546.88 KiB | 14.39 MiB/s, done.
Resolving deltas: 100% (754/754), done.
Already up to date.


In [5]:
!pip install -r environment/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 717.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [6]:
import sys
# Add src to path
sys.path.append(os.path.join(os.getcwd(), 'src'))

# Define Drive paths
DRIVE_DATA_PATH = '/content/drive/MyDrive/Thesis_DNN_Planner/data'
DRIVE_MODELS_PATH = '/content/drive/MyDrive/Thesis_DNN_Planner/models'
DRIVE_RESULTS_PATH = '/content/drive/MyDrive/Thesis_DNN_Planner/results'

# Link Drive storage to local cloned directories
if not os.path.islink('data'):
    !ln -s {DRIVE_DATA_PATH} data
if not os.path.islink('models'):
    !ln -s {DRIVE_MODELS_PATH} models
if not os.path.islink('results'):
    !ln -s {DRIVE_RESULTS_PATH} results

print("✅ Environment setup complete.")

✅ Environment setup complete.


# Comparative Inference Analysis

In [7]:

from pathlib import Path
import numpy as np
from torch.utils.data import DataLoader

from dnn_guidance.data_loader import PathfindingDataset
from dnn_guidance.inference import InferenceHandler
from dnn_guidance.model import UNetFiLM, HRFiLMNet, ResNetFPNFiLM
from utils.visualization import plot_inference_comparison


In [8]:
unet_class, unet_ckpt = UNetFiLM, DRIVE_RESULTS_PATH / Path('checkpoints/unet_film_v2_dice_heavy/unet_film_v2_dice_heavy_best_model.pth')
hr_class, hr_ckpt = HRFiLMNet, DRIVE_RESULTS_PATH / Path('checkpoints/hr_film_net_v1_baseline/hr_film_net_v1_baseline_best_model.pth')
resnet_class, resnet_ckpt = ResNetFPNFiLM, DRIVE_RESULTS_PATH / Path('checkpoints/resnet_fpn_film_v2/resnet_fpn_film_v2_best_model.pth')

MODEL_REGISTRY = {
    'UNet-FiLM': {'model_class': unet_class, 'checkpoint_path': hr_class},
    'HR-FiLM-Net': {'model_class': hr_class, 'checkpoint_path': hr_class},
    'ResNet-FPN-FiLM': {'model_class': resnet_class, 'checkpoint_path': resnet_class},
    'HR-FiLM-Net | ResNet-FPN-FiLM': [
        {'model_class': hr_class, 'checkpoint_path': hr_class, 'weight': 1},
        {'model_class': resnet_class, 'checkpoint_path': resnet_class, 'weight': 0.5},
    ]
}

In [9]:
# Create test DataLoader
samples_dir = DRIVE_DATA_PATH / Path('training_data')
gt_dir = DRIVE_DATA_PATH / Path('ground_truth')
dataset = PathfindingDataset(samples_dir, gt_dir, augment=False)
loader = DataLoader(dataset, batch_size=1, shuffle=False)


In [10]:
num_samples_to_visualize = 5
random_indices = np.random.choice(len(dataset), num_samples_to_visualize, replace=False)

for i in random_indices:
    grid, robot, gt_heatmap = dataset.get_raw_item(i)
    predictions_to_plot = {}
    for name, model_config in MODEL_REGISTRY.items():
        handler = InferenceHandler(model_config, device='cpu')
        heatmap = handler.predict(grid, robot)
        predictions_to_plot[name] = heatmap
    plot_inference_comparison({
        'input_grid': grid,
        'ground_truth_heatmap': gt_heatmap,
        'predictions': predictions_to_plot,
    })

TypeError: expected str, bytes or os.PathLike object, not type

Analysis notes go here.